In [ ]:
import requests

def call_hdf5():
    try:
        # Make a GET request to the hdf5 service
        response = requests.get("http://localhost:5000/read")  # Replace "/endpoint" with the actual endpoint of your Flask app
        
        # Check if the request was successful
        if response.status_code == 200:
            return {"status": "success", "data": response.json()}
        else:
            return {"status": "error", "message": f"HTTP {response.status_code}: {response.text}"}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": str(e)}

In [ ]:
print(call_hdf5())

In [2]:
# Import the module from sgs_core
import sys
from pathlib import Path

# Add the sgs_core directory to the Python path
sys.path.append(str(Path.cwd() / "sgs_core"))

# Import the meta_algebra module
import meta_algebra as md

hll1 = md.HllSet(6)
hll2 = md.HllSet(6)

hll1.add("apple")
hll1.add("banana")
hll1.add("cherry")

hll2.add("banana")
hll2.add("date")
hll2.add("elderberry")

# Perform union and intersection
result_union = hll1.union(hll2)
result_intersection = hll1.intersection(hll2)

print("Union:", result_union.count())
print("Intersection:", result_intersection.count())

complement = hll1.complement(hll2)
print("Complement:", complement.count())
d, r, n = hll1.difference(hll2)
print("Difference:", d.count(), r.count(), n.count())

Union: 5
Intersection: 1
Complement: 2
Difference: 2 1 2


In [ ]:
hll3 = md.HllSet(6)
hll2.to_binary_tensor()

In [ ]:
import redis
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
# from redis.commands.search.client import Client

class MetaRedis:
    def __init__(self, host='redis', port=6379, db=0):
        """
        Initialize a connection to Redis.
        
        Args:
            host: Redis host (default 'redis')
            port: Redis port (default 6379)
            db: Redis database number (default 0)
        """
        self.redis = redis.Redis(
            host=host,
            port=port,
            db=db,
            socket_connect_timeout=5,  # 5 second timeout
            decode_responses=False  # Keep binary data intact
        )

        try:
            self._create_indices()
            print("Redisearch indices created successfully.")
        except redis.exceptions.ResponseError as e:
            if "Index already exists" in str(e):
                print("Indices already exist.")
            else:
                print(f"Error creating indices: {e}")
        except redis.exceptions.ConnectionError:
            print("Could not connect to Redis")
        except redis.exceptions.TimeoutError:
            print("Redis connection timed out")        
        except redis.exceptions.RedisError as e:
            print(f"Redis error: {e}")
        
    def _create_indices(self):
        """Initialize Redisearch indices."""
        # Edges Index (left_hll → right_hll)
        self.redis.ft("edges").create_index([
            TextField("e_sha1"),
            TextField("label"),
            TextField("left"),
            TextField("right"),
            TextField("attr")
        ], definition=IndexDefinition(prefix=["meta:edges:"]))

        # Tokens Index (token → data_ids)
        self.redis.ft("tokens").create_index([
            TextField("hash"),
            NumericField("bin"),
            NumericField("zeros"),
            NumericField("TF"),
            TagField("refs")
        ], definition=IndexDefinition(prefix=["meta:tokens:"]))

        # Commits Index (commit → timestamp)
        self.redis.ft("commits").create_index([
            TextField("c_sha1"),
            NumericField("timestamp")
        ], definition=IndexDefinition(prefix=["meta:commits:"]))    

    
    def execute_graph_query(self, query):
        """Execute a Redis Graph query."""
        return self.redis.execute_command("GRAPH.QUERY", "graph", query)

    def roaring_bitmap_command(self, command, *args):
        """Execute a Redis Roaring Bitmap command."""
        return self.redis.execute_command(command, *args)

    def search(self, index_name, query):
        """Search a Redisearch index."""
        return self.redisearch_client.search(query)

# Example usage
if __name__ == "__main__":
    meta_redis = MetaRedis(host="localhost", port=6379)
    print("Redis connection initialized and indices ensured.")

In [ ]:
meta_redis = MetaRedis(host="localhost", port=6379)

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI(api_key="sk-4238636dfa9f48269e13602659a9eea4", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Привет, как дела?"},
    ],
    stream=False
)

print(response.choices[0].message.content)